# Evaluate finetuned Whisper on Fleurs

## Reproduce reported results to check validity

check text tokenizer

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from datasets import load_dataset
# yue_hant_hk for Cantonese and cmn_hans_cn for Chinese
dataset = load_dataset("google/fleurs", "cmn_hans_cn", split='test')

In [8]:
print(dataset)
# print(dataset[0])
dataset = dataset.remove_columns(['id', 'num_samples', 'path', 'gender', 'lang_id','language','lang_group_id'])
print(dataset[0])

Dataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 945
})
{'audio': {'path': '11338778690242874435.wav', 'array': array([ 4.17232513e-07, -5.36441803e-07,  5.96046448e-07, ...,
       -1.31678581e-03, -1.24770403e-03, -1.29789114e-03]), 'sampling_rate': 16000}, 'transcription': '1940 年 8 月 15 日 盟 军 攻 入 法 国 南 部 这 次 进 攻 被 称 为 龙 骑 兵 行 动', 'raw_transcription': '1940 年 8 月 15 日，盟军攻入法国南部，这次进攻被称为“龙骑兵行动”。'}


In [9]:
# tokenizer
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Chinese", task="transcribe")
# processor
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Chinese", task="transcribe")

In [10]:
input_str = dataset[0]["raw_transcription"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 1940 年 8 月 15 日，盟军攻入法国南部，这次进攻被称为“龙骑兵行动”。
Decoded w/ special:    <|startoftranscript|><|zh|><|transcribe|><|notimestamps|>1940 年 8 月 15 日，盟军攻入法国南部，这次进攻被称为“龙骑兵行动”。<|endoftext|>
Decoded w/out special: 1940 年 8 月 15 日，盟军攻入法国南部，这次进攻被称为“龙骑兵行动”。
Are equal:             True


check base model on Chinese (simplified and cantonese)

In [13]:
from jiwer import wer
from whisper.normalizers import BasicTextNormalizer
import re
import unicodedata

normalizer = BasicTextNormalizer()
def map_to_pred(batch):
    sampling_rate = batch.features["audio"].sampling_rate
    input_features = processor(batch["audio"][0]["array"], sampling_rate=sampling_rate, return_tensors="pt").input_features
    generated_ids = model.generate(inputs=input_features.to("cuda"))
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True,
                                           # normalize=True
                                           )
    # batch["logits"] = generated_ids.cpu().detach().numpy()
    batch['prediction'] = [custom_normalizer(transcription[0], "zh")]
    # batch['prediction'] = transcription[0]
    # batch['ground_truth'] = processor.tokenizer._normalize(batch['transcription'][0])
    # batch['ground_truth'] = [normalizer(batch['transcription'][0])]
    # batch['ground_truth'] = batch['transcription']
    # print(batch)
    return batch

def custom_normalizer(text, lang):
    """
    normalizing procedures based on appendix C, Whisper OpenAI paper
    language tokens based on https://github.com/openai/whisper/blob/main/whisper/tokenizer.py
    """
    if lang == 'en':
        return normalizer(text)
    else:
        text = re.sub("[\(\[].*?[\)\]]", "", text) # removes [] and () as well as content in-between -- will not work for non-standard brackets, eg: <> or （）, etc
        text = unicodedata.normalize("NFKC", text)
        ch_text = []
        for ch in text:
            if unicodedata.category(ch)[0] not in ('M', 'P', 'S'):
                ch_text.append(ch)
            else:
                ch_text.append(' ')
        text = ''.join(ch_text)
        text = text.lower()
    # set up for character error rate for languages w/o spaces between words
    if lang in ('zh', 'ja', 'th', 'lo', 'my'):
        text = ' '.join(text)
        text = re.sub('(?<=\d) (?=\d)', '', text)
    return re.sub(' +', ' ', text)

In [34]:
%%time
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base").to("cuda")
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
# using "zh" in model => traditional chinese whereas Fleurs uses simplified Chinese for cmn_hans_cn, and yue_hant_hk
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language = "zh", task = "transcribe")
result = dataset.map(map_to_pred, batched=True, batch_size = 1)


  0%|          | 0/945 [00:00<?, ?ba/s]

In [38]:
print("WER:", wer(result["ground_truth"], result["prediction"])) # (truth, pred)

WER: 0.38239503090460675


In [36]:
print(result[0])

{'audio': {'path': None, 'array': array([ 0.00000000e+00, -3.05175781e-05,  0.00000000e+00, ...,
       -1.34277344e-03, -1.25122070e-03, -1.31225586e-03]), 'sampling_rate': 16000}, 'transcription': '1940 年 8 月 15 日 盟 军 攻 入 法 国 南 部 这 次 进 攻 被 称 为 龙 骑 兵 行 动', 'raw_transcription': '1940 年 8 月 15 日，盟军攻入法国南部，这次进攻被称为“龙骑兵行动”。', 'prediction': '1940 年 8 月 15 日 蒙 軍 公 路 法 國 南 部 這 次 進 攻 被 稱 為 龍 旗 冰 晴 洞', 'ground_truth': '1940 年 8 月 15 日 盟 军 攻 入 法 国 南 部 这 次 进 攻 被 称 为 龙 骑 兵 行 动'}


In [15]:
from datasets import load_dataset
# yue_hant_hk for Cantonese and cmn_hans_cn for Chinese
dataset = load_dataset("google/fleurs", "yue_hant_hk", split='test')

Found cached dataset fleurs (/home/sivan/.cache/huggingface/datasets/google___fleurs/yue_hant_hk/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)


In [16]:
print(dataset)
# print(dataset[0])
dataset = dataset.remove_columns(['id', 'num_samples', 'path', 'gender', 'lang_id','language','lang_group_id', 'raw_transcription'])
print(dataset[0])

Dataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 819
})
{'audio': {'path': '4705644860951940224.wav', 'array': array([ 0.        ,  0.        ,  0.        , ...,  0.00025684,
       -0.00013679, -0.00051916]), 'sampling_rate': 16000}, 'transcription': '仍 有 許 多 當 時 在 此 的 男 女 存 活 了 下 來 還 有 更 多 人 的 摯 愛 在 此 被 殺 害 或 勞 動 至 死 不 管 是 不 是 猶 太'}


In [24]:
%%time
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base").to("cuda")
processor = WhisperProcessor.from_pretrained("openai/whisper-base")
# using "zh" in model => traditional chinese whereas Fleurs uses simplified Chinese for cmn_hans_cn, and yue_hant_hk
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language = "zh", task = "transcribe")
result = dataset.map(map_to_pred, batched=True, remove_columns=['audio'], batch_size = 1)


  0%|          | 0/819 [00:00<?, ?ba/s]

CPU times: user 6min 57s, sys: 14.3 s, total: 7min 11s
Wall time: 4min 34s


In [26]:
print("WER:", wer(result["transcription"], result["prediction"])) # (truth, pred)

WER: 0.4356259136449531


In [25]:
print(result[0])

{'transcription': '仍 有 許 多 當 時 在 此 的 男 女 存 活 了 下 來 還 有 更 多 人 的 摯 愛 在 此 被 殺 害 或 勞 動 至 死 不 管 是 不 是 猶 太', 'prediction': '仍 有 許 多 當 時 在 此 的 男 女 徐 樂 了 下 來 還 有 更 多 人 的 自 愛 在 此 被 殺 害 惑 努 動 之 死 不 過 是 不 事 由 太 '}
